In [1]:
import pandas as pd
import numpy as np

movies = pd.read_csv("ml-25m/movies.csv") 
ratings = pd.read_csv("ml-25m/ratings.csv") 

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
#Filter data to include movies with at least 14500 ratings and users with at least 250 ratings

data = ratings.groupby('movieId').filter(lambda x: len(x) >= 14500)
data = data.groupby('userId').filter(lambda x: len(x) >= 250)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 407450 entries, 51820 to 24996197
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     407450 non-null  int64  
 1   movieId    407450 non-null  int64  
 2   rating     407450 non-null  float64
 3   timestamp  407450 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 15.5 MB


In [3]:
#Calculate the average ratings for each movie and add it to a new dataset

avg_data = data.groupby('movieId')[['movieId','rating']]. mean()

In [4]:
#View movies and their average ratings in descending order

avg_data.sort_values('rating', ascending=False)

,movieId,rating
movieId,,
318,318,4.415127
858,858,4.404301
296,296,4.402373
1196,1196,4.314599
1198,1198,4.310673
...,...,...
185,185,2.513812
788,788,2.473963
153,153,2.450936


In [5]:
#Append the averaage rating onto the movies dataset

movies['avg_rating'] = avg_data['rating']

In [6]:
#Group the data in the same order by movie average ratings and merge the datasets to contain movieId, title, genres, 
#and average ratings

avg_data2 = data.groupby('movieId')[['rating']].mean()
movies_ratings = pd.merge(left=movies, right=avg_data2, left_on='movieId', right_on='movieId')
movies_ratings = movies_ratings[['movieId', 'title', 'genres', 'rating']]

In [7]:
#Sort the movies in descending order according to their average ratings

movies_ratings = movies_ratings.sort_values('rating', ascending=False)

In [19]:
#Shows the movies recommended in descending order like a recommender system would

movies_ratings[['title', 'genres', 'rating']]

,title,genres,rating
42,"Shawshank Redemption, The (1994)",Crime|Drama,4.415127
92,"Godfather, The (1972)",Crime|Drama,4.404301
38,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.402373
117,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,4.314599
119,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,4.310673
...,...,...,...
27,"Net, The (1995)",Action|Crime|Thriller,2.513812
91,"Nutty Professor, The (1996)",Comedy|Fantasy|Romance|Sci-Fi,2.473963
23,Batman Forever (1995),Action|Adventure|Comedy|Crime,2.450936
7,Ace Ventura: When Nature Calls (1995),Comedy,2.392732


In [9]:
#Create a list of the top 25 most popular movies

movies_ratings_indices = movies_ratings['movieId'][0:25]

In [10]:
#Create a list of all unique users

userID_list = data['userId'].unique()

In [20]:
#Compute average precision, recall, and F1 score

precision_matrix = np.zeros(shape=(len(userID_list)))
recall_matrix = np.zeros(shape=(len(userID_list)))
f1_matrix = np.zeros(shape=(len(userID_list)))

for idx,i in enumerate(userID_list):
    userdf = data[data['userId']==i]
    top_movies_rated = userdf[userdf['rating']>=3.5]

    top_rated_indices = top_movies_rated['movieId'].values
    
    tp = len(np.intersect1d(movies_ratings_indices, top_rated_indices))
    fp = len(movies_ratings_indices)-tp
    fn = len(top_rated_indices)-tp

    precision = tp / (tp + fp + 1e-10)
    precision_matrix[idx] = precision
    
    recall = tp / (tp + fn + 1e-10)
    recall_matrix[idx] = recall

    f1 = 2.0 * (precision * recall) / (precision + recall + 1e-10)
    f1_matrix[idx] = f1


avg_precision = np.average(precision_matrix)
avg_recall = np.average(recall_matrix)
avg_f1 = np.average(f1_matrix)
print('Average Precision: ', avg_precision)
print('Average Recall: ', avg_recall)
print('Average F1 Score: ', avg_f1)

Average Precision:  0.8141970310358797
Average Recall:  0.11373786361472962
Average F1 Score:  0.19757752514916344
